### CNN using Fast.ai

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
# This file contains all the main external libs we'll use
from fastai.imports import *
# Normal imports
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
PATH = "data/dogscats/"
sz = 224 # the size the images will be resized to

Train > Cats
      > Dogs
Valid > Cats
      > Dogs

Here the objects to be classified are called classes <br\>
We will be using the <b>resnet34 model</b> for our image classifier

In [ ]:
arch = resnet34
data = ImageClassifierData.from_paths(PATH,tfms=tfms_from_model(arch,sz)) # tfms=tfms_from_model this takes care of 
#transforming the data like resizing, normalizing
learn = ConvLearner.pretrained(arch, data, precompute=True) # this is a learner based on 1000's of data
learn.fit(0.01,2) # 0.01 is the learning rate and 2 is the number of epochs

### Setting the learning rate
It is how quickly or slowly you update the weights of the model.It has to be optimum.<br\>
We are dealing with the final frozen layer


In [ ]:
# Create a new learner
learn = ConvLearner.pretrained(arch,data,precompute = True)
lrf = learn.lr_find() # lr_find method will give you the learning rate

After the model has been fit, we can use the attribute sched to find the value

In [ ]:
learn.sched.plot()
# The point where the loss stong decreasing should be our learning rate

### Data augmentation & SGDR
It is a way to create more data by augmenting your existing training data by zooming,rotating it.

In [ ]:
tfms = tms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
#the above function will transform the data , aug_tfms=transforms_side_on will flip the images along the vertical
#axis and max zoom will zoom the images for +_10per

### Creating a new data for the augmented set

In [ ]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms)
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 1, cycle_len = 1)
# Number of epochs after which the cycle is reset. For SGDR (Stochastic Gradient Desecent
#with Reset)

### Saving the model

In [ ]:
learn.save('224_lastlayer')
learn.load('224_lastlayer')

At this stage the weightage of the final layer has been calculated.

Now we have to optimize the values of the other layers

In [ ]:
learn.unfreeze()

### Differential layer rates
This is done to finetune the layers

In [ ]:
lr = np.array([1e-4,1e-3,1e-2])
learn.fit(lr, 3, cycle_len=1, cycle_mult=2) #cycle_mult?
learn.save('224_all')
learn.load('224_all')

### TTA 
Test time augmentaion is the process of applying augmentation at the time of testing, this will increase the accuracy by 10-20%

In [ ]:
log_pred , y = learn.TTA()
probs = np.mean(np.exp(log_pred),0)

In [ ]:
accuracy_np(probs,y)

### Final code

In [ ]:
tfms = tms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms)
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 1, cycle_len = 1)

learn.unfreeze()
lr = np.array([1e-4,1e-3,1e-2])
learn.fit(lr, 3, cycle_len=1, cycle_mult=2) #cycle_mult?

log_pred , y = learn.TTA()
probs = np.mean(np.exp(log_pred),0)
accuracy_np(probs,y)